In [3]:
import sys
!{sys.executable} -m ensurepip --upgrade

Looking in links: /tmp/tmp5qy0tkkh


In [4]:
%pip install ipython-sql
%pip install prettytable==3.11.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%load_ext sql
%sql sqlite:///myex.db
%config SqlMagic.autopandas=False
%config SqlMagic.displaycon=False

In [6]:
def tables():
    tables = %sql select tbl_name from sqlite_master where type="table" or type="view"
    return list(tables)

def schema(tbl_name="*"):
    schema = %sql select sql from sqlite_master where type="table" and tbl_name="{tbl_name}"
    return schema

In [7]:
%%sql
drop table if exists 注文;
drop table if exists 座席;
drop table if exists メニュー;

create table メニュー(
    メニュー番号 char(3) not null,
    メニュー名 varchar(30),
    価格 int,
    原価 int,
    primary key(メニュー番号)
);

create table 座席(
    座席番号 char(3) not null,
    種別 varchar(20),
    喫煙 varchar(10),
    primary key(座席番号)
);

create table 注文(
    注文番号 int not null,
    座席番号 char(3),
    メニュー番号 char(3),
    時刻 real,
    primary key(注文番号),
    foreign key(座席番号) references 座席(座席番号),
    foreign key(メニュー番号) references メニュー(メニュー番号)
);

Done.
Done.
Done.
Done.
Done.
Done.


[]

In [8]:
%%sql
insert into メニュー values('M01', 'ライトセット１', 300, 100);
insert into メニュー values('M02', 'ライトセット２', NULL, 250);
insert into メニュー values('M11', 'セット１', 650, 200);
insert into メニュー values('M12', 'セット１', 650, 250);
insert into メニュー values('M20', 'スペシャルセット', NULL, 850);

/* 座席データの挿入 */
insert into 座席 values('001', 'カウンター', '不可');
insert into 座席 values('002', 'テーブル', '不可');
insert into 座席 values('003', 'テーブル', '可');
insert into 座席 values('004', 'テーブル', '可');

/* 注文データの挿入 */
insert into 注文 values(1, '002', 'M12', 10.75);
insert into 注文 values(2, '001', 'M20', 11.25);
insert into 注文 values(3, '004', 'M20', 11.75);
insert into 注文 values(4, '001', 'M12', 12.25);
insert into 注文 values(5, '003', 'M01', 12.75);
insert into 注文 values(6, '002', 'M11', 13.25);
insert into 注文 values(7, '001', 'M20', 14.00);
insert into 注文 values(8, '004', 'M02', 14.50);

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [9]:
%%sql

update メニュー
set 価格 = 原価 + 200
where 価格 is null;

update メニュー
set 価格 = 価格 * 1.2;

2 rows affected.
5 rows affected.


[]

In [10]:
%%sql

select * from メニュー;

Done.


メニュー番号,メニュー名,価格,原価
M01,ライトセット１,360,100
M02,ライトセット２,540,250
M11,セット１,780,200
M12,セット１,780,250
M20,スペシャルセット,1260,850


In [ ]:
%%sql
drop view if exists 低原価率メニュー一覧; /

create view 低原価率メニュー一覧 as
select メニュー番号, 価格, 原価
from メニュー
where 原価 <= 価格 * 0.3;

select メニュー番号
from 注文
where 時刻 >= 12
  and 座席番号 in ('002', '003', '004');

Done.
Done.
Done.


メニュー番号
M01
M11
M02


In [16]:
%%sql

select * from 低原価率メニュー一覧;

Done.


メニュー番号,価格,原価
M01,360,100
M11,780,200
